In [1]:
# Bibiliotecas;
import pandas as pd 

from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

In [2]:
# Load dados;
data = pd.read_csv("../Databases/autos.csv", encoding="ISO-8859-1")

## Pre-processamento

In [3]:
data.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [4]:
# Removendo coluna de dados que não estão relacionadas com o veiculo;
data = data.drop(['dateCrawled','name','lastSeen','seller',
                  'postalCode','dateCreated','nrOfPictures', 
                  'offerType'], axis=1)

In [42]:
# Fazendo filtragem de dados;
data = data[(data.price > 100) & (data.price < 350000)]

# Valores nulos, verificando existencia;
data.loc[pd.isnull(data["vehicleType"])] 
data.loc[pd.isnull(data["gearbox"])]
data.loc[pd.isnull(data["model"])] 
data.loc[pd.isnull(data["fuelType"])]
data.loc[pd.isnull(data["notRepairedDamage"])] 

# Dicionario de substituição, baseado no valor que mais se repete;
values = {"vehicleType": data["vehicleType"].value_counts().index[0], # limousine
          "gearbox": data["gearbox"].value_counts().index[0], # manuell
          "model": data["model"].value_counts().index[0], # golf     
          "fuelType": data["fuelType"].value_counts().index[0], # benzin
          "notRepairedDamage": data["notRepairedDamage"].value_counts().index[0]} # nein

# Filtro para valors nulos;
data = data.fillna(values)